In [33]:
# Pandas is used for data manipulation
import pandas as pd


## RF on SOCS model using python -- Huang et al., 2019

### Perform RF

In [34]:
from google.colab import drive
drive.mount('/content/drive')

path='/content/drive/Shareddrives/DS Projects/Jingyi Huang-Cyber Infrastructure/From Jingyi/CI-submission&RFmodel/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import pandas as pd
import numpy as np

# Read the data in CSV format
soil_data = pd.read_csv(path+'Data/soil.txt', header=0, sep=",")

soil_data = soil_data.dropna()
soil_df = soil_data.drop(soil_data.columns[[0, 1, 2, 3, 16, 19]], axis=1)


# Convert column 16 (now column 12 after dropping columns) to categorical type
soil_df.iloc[:, 12] = soil_df.iloc[:, 12].astype('category')

soil_df['lc2'] = 'lc'

# create a new column 'Land_Cover' based on conditions
conditions = [
    (soil_df.iloc[:, 15] == 8),
    (soil_df.iloc[:, 15] == 9),
    (soil_df.iloc[:, 15] == 10),
    (soil_df.iloc[:, 15] == 11),
    (soil_df.iloc[:, 15] == 13),
    (soil_df.iloc[:, 15] == 14),
    (soil_df.iloc[:, 15] == 15),
    (soil_df.iloc[:, 15] == 16)
]

values = [
    'Forest',
    'Forest',
    'Forest',
    'Grassland',
    'Cropland',
    'Pasture',
    'Wetland',
    'Wetland'
]

soil_df['lc2'] = np.select(conditions, values, default='Unknown')

In [36]:
soil_df.groupby('lc2').count()

,SOC,Depth,dem,slope,aspect,hillshade,twi,mrvbf,clay,silt,sand,pH,tmax,tmin,prcp,lc
lc2,,,,,,,,,,,,,,,,
Cropland,658,658,658,658,658,658,658,658,658,658,658,658,658,658,658,658
Forest,1528,1528,1528,1528,1528,1528,1528,1528,1528,1528,1528,1528,1528,1528,1528,1528
Grassland,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12
Pasture,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649,649
Wetland,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94,94


In [37]:
len(soil_df)

2941

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming soil_data is your original DataFrame

# Get the number of unique Soil_IDs
nlength = soil_data['Soil_ID'].nunique()

# Split the data into calibration (75%) and validation (25%) sets
train_data, val_data = train_test_split(soil_data, test_size=0.25, random_state=111, stratify=soil_data['Soil_ID'])

# Create calibration and validation DataFrames
cali = train_data#[['Soil_ID', 'lc2', 'Land_Cover']]
vali = val_data#[['Soil_ID', 'lc2', 'Land_Cover']]

# Print the summary of the calibration set
print("Calibration set:")
print(len(cali))

# Print the summary of the validation set
print("\nValidation set:")
print(len(vali))

Calibration set:
2205

Validation set:
736


In [40]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Define the features and target variable
features = ['Depth', 'tmax', 'tmin', 'prcp', 'lc', 'clay', 'silt', 'sand', 'dem', 'slope', 'aspect', 'hillshade', 'twi', 'mrvbf']
target = 'SOC'

# Extract features and target variable for calibration set
X_cali = cali[features]
y_cali = cali[target]

# Train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=500, max_features=10, random_state=42)
rf_model.fit(X_cali, y_cali)

# Predict on the calibration data
rf_predict = rf_model.predict(X_cali)

# Calculate goodness of fit
gof_rf_predict = r2_score(y_cali, rf_predict)

# Calibration
print("Goodness of fit:", gof_rf_predict)


Goodness of fit: 0.8931929486830533


### Saving the model

In [41]:
path

'/content/drive/Shareddrives/DS Projects/Jingyi Huang-Cyber Infrastructure/From Jingyi/CI-submission&RFmodel/'

In [45]:
from joblib import dump

# Saving the model
dump(rf_model, path+'rf_model.joblib')


['/content/drive/Shareddrives/DS Projects/Jingyi Huang-Cyber Infrastructure/From Jingyi/CI-submission&RFmodel/rf_model.joblib']

### Testing the pickle and API

In [59]:
example1 = {"Depth": 5,
            "tmax": 10.702739716,
            "tmin": 0.5561643839,
            "prcp": 753.0,
            "lc": 9.0,
            "clay": 10.0,
            "silt": 35.0,
            "sand": 55.0,
            "dem": 189,
            "slope": 5.69661e-05,
            "aspect": 6.283185482,
            "hillshade": 0.7853578925,
            "twi": 11.223488808,
            "mrvbf": 2.5688176155
      }

In [ ]:
## TEsting the pickle model

features = ['Depth', 'tmax', 'tmin', 'prcp', 'lc', 'clay', 'silt', 'sand', 'dem', 'slope', 'aspect', 'hillshade', 'twi', 'mrvbf']
#dictionary = {key: [value] for key, value in class_model.dict().items()}
df = pd.DataFrame([example])

print(df)

new=rf_model.predict(df[features])[0]
print(new)

   Depth      tmax      tmin   prcp   lc  clay  silt  sand  dem     slope  \
0      5  10.70274  0.556164  753.0  9.0  10.0  35.0  55.0  189  0.000057   

     aspect  hillshade        twi     mrvbf  
0  6.283185   0.785358  11.223489  2.568818  
5.626214404736811


### API CALLS

In [97]:
# Maybe customize this url, an option "https://connect.doit.wisc.edu/content/jhuang_socs_forecasting"
url_path="https://connect.doit.wisc.edu/content/37ea4362-968a-4899-bc03-244f4d92d826"

In [98]:
import requests
import json
import warnings
warnings.filterwarnings("ignore", message="Unverified HTTPS request is being made.*")

connect_api_key = 'XXXXX'

In [99]:
def api_request(data):
  data_json = json.dumps(data)

  headers = {
    "Authorization": f"Key {connect_api_key}",
    "accept": "application/json",
    "Content-Type": "application/json"
  }

  request_url = url_path+"/v1/prediction"
  #build_url(connect_server, url_path)

  # Make the POST request with verification disabled
  response = requests.post(request_url, headers=headers, data=data_json, verify=False)

  # Response
  print(f"Status Code: {response.status_code}")
  try:
      response_json = response.json()
      print(response_json)
  except json.JSONDecodeError:
      print("Response content is not in JSON format.")
      print(response.text)

#### Example 1

In [100]:
#2137
data = {
    #"SOC": 13.19165618,
    "Depth": 5,"tmax":11.426027,
    "tmin": 0.17808220,"prcp": 633,"lc": 15,"clay": 12,
    "silt": 20,
    "sand": 68,
    "dem": 295,
    "slope": 0.0000357995,
    "aspect": 5.8894872665,
    "hillshade": 0.7853651,
    "twi": 12.869823,
    "mrvbf": 7.093913e+00
} #prediction 7.98289171

print("The prediction from R output is: ", 7.98289171)
print("The prediction from python implementation is: ")
api_request(data)

The prediction from R output is:  7.98289171
The prediction from python implementation is: 
Status Code: 200
{'soil_organic_carbon_stock': 9.345605187969879}


#### Example 2

In [86]:
#2735 from validation
data = {
    #"SOC": 13.19165618,
    "Depth": 15,"tmax":12.865753,
    "tmin": 1.56712329,"prcp": 1052,"lc": 8,"clay": 14,
    "silt": 66,
    "sand": 20,
    "dem": 295,
    "slope": 0.2205786258,
    "aspect": 5.4452042580,
    "hillshade": 0.5652189,
    "twi": 6.078456,
    "mrvbf": 4.363611e-03
}

In [87]:
print("The prediction from R output is: ", 1.27211851)
print("The prediction from python implementation is: ")
api_request(data)

The prediction from R output is:  1.27211851
The prediction from python implementation is: 
Status Code: 200
{'soil_organic_carbon_stock': 1.2558310193539057}


####. Null example

In [68]:
#print("The prediction from R output is: ", 1.27211851)
print("The prediction from python implementation is: ")
api_request(data)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'connect.doit.wisc.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'connect.doit.wisc.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Status Code: 200
{'soil_organic_carbon_stock': 5.626214404736811}


#### Just testing

In [88]:
data = {
      "Depth": 60,
      "tmax": 12.42,
      "tmin": 0.45,
      "prcp": 954,
      "lc": 13,
      "clay": 2.0,
      "silt": 10.0,
      "sand": 88.0,
      "dem": 331,
      "slope": 0.03,
      "aspect": 6.28,
      "hillshade": 0.7588,
      "twi": 5.87,
      "mrvbf": 4.72
}

api_request(data)

Status Code: 200
{'soil_organic_carbon_stock': 0.15928801590680008}
